# Instalando bibliotecas

In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import openpyxl
import time


# Inicializando navegador


In [37]:
options = Options()
# options.add_argument("--headless")  # Executa sem abrir janela
options.add_argument("--no-sandbox")

# Aponta para o chromedriver na raiz do projeto
service = Service(executable_path="./chromedriver-linux64/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

# Acessa a página dos campeões
driver.get('https://www.op.gg/champions')

# Inicializando a raspagem


In [38]:
# Scroll infinito até o fim da página
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Raspando os dados


In [61]:
from selenium.webdriver.common.by import By
import pandas as pd
import time  # para dar tempo da página atualizar após cada clique

# Lanes que vamos processar
lanes = ["Top", "Jungle", "Middle", "Bottom", "Support"]

# Lista para juntar todos os dados
all_data = []

for lane in lanes:
    # Clica no botão da lane
    spans = driver.find_elements(By.TAG_NAME, "span")
    for span in spans:
        if span.text.strip().lower() == lane.lower():
            span.click()
            time.sleep(2)  # Aguarda a tabela atualizar (ajuste se necessário)
            break

    # Raspa os dados da tabela
    try:
        tbody = driver.find_element(By.TAG_NAME, 'tbody')
        rows = tbody.find_elements(By.TAG_NAME, 'tr')
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, 'td')
            row_data = [cell.text for cell in cells]
            if row_data:
                # Adiciona a lane como uma nova coluna no início
                all_data.append([lane] + row_data)
    except:
        print(f"Erro ao processar lane {lane}")

# Verifica se as linhas têm a mesma quantidade de colunas
print(set(len(row) for row in all_data))

# Ajuste os nomes das colunas conforme a estrutura real da tabela
# Aqui estou assumindo 8 colunas de dados + 1 coluna da lane
columns = ['Lane', '', 'Champion', '', '', 'Win Rate', 'Pick Rate', '', '']  # ajuste conforme necessário

# Filtra linhas inconsistentes
all_data = [row for row in all_data if len(row) == len(columns)]

# Cria o DataFrame
df = pd.DataFrame(all_data, columns=columns)

# Exporta para Excel
df.to_excel("todos_champions.xlsx", index=False)


{8, 1}


<selenium.webdriver.remote.webelement.WebElement (session="bfa394dcd740f39a2a491fa4305b4a7f", element="f.C58E1A26C5B106833B26C2B20F707037.d.EA317CC4E2D277A8FC83419F22890184.e.2322")>

In [56]:
data[19]

['20', 'Kayn', '', '', '51.68%', '8.58%', '', '']